In [57]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from collections import defaultdict

In [108]:
domain = "jobs"

In [109]:
df = pd.read_excel(f"../pre_experiment_scripts/{domain}_explanations.xlsx", dtype={"code 0": str,
                                                                                   "code 1": str})
# df = df.drop(columns = ["Unnamed: 0"])

In [110]:
df.head()

,code 0,code 1,prompt A,prompt B,explanation A,explanation B
0,000000,000001,Generate an explanation with the following dim...,Generate an explanation with the following dim...,```html\n<h3>Recommended Role: Cashier</h3>\nT...,```html\n<h3>Recommended Role: Cashier</h3>\nY...
1,000000,000010,Generate an explanation with the following dim...,Generate an explanation with the following dim...,```html\n<h3>Recommended Role: Cashier</h3>\nT...,```html\n<h3>Recommended Role: Cashier</h3>\nY...
2,000000,000100,Generate an explanation with the following dim...,Generate an explanation with the following dim...,```html\n<h3>Recommended Role: Cashier</h3>\nT...,```html\n<h3>Recommended Role: Cashier</h3>\nH...
3,000000,001000,Generate an explanation with the following dim...,Generate an explanation with the following dim...,```html\n<h3>Recommended Role: Cashier</h3>\nT...,```html\n<h3>Recommended Role: Cashier</h3>\n<...
4,000000,010000,Generate an explanation with the following dim...,Generate an explanation with the following dim...,```html\n<h3>Recommended Role: Cashier</h3>\nT...,```html\n<h3>Recommended Role: Cashier</h3>\nT...


In [111]:
new_df = defaultdict(list)

def code_to_list(row):

    row_dict = defaultdict(list)
    
    features = ["Domain", "Length", "Structure", "Formality", "Detail", "Persuasiveness"]
    flipped = ["Formality", "Detail", "Persuasiveness"]

    mapping = {"Domain" : {"1": "High-stakes", "0": "Low-stakes"},
               "Length" : {"1": "Long", "0": "Short"},
               "Structure" : {"1": "Bulleted", "0": "Running"},
               "Formality" : {"1": "Informal", "0": "Formal"},
               "Detail" : {"1": "Aggregated", "0": "Detailed"},
               "Persuasiveness": {"1": "Decision-support", "0": "Persuasive"}}

    row_dict["Code"].append(row["code 0"])
    row_dict["Explanation"].append(row["explanation A"])
        
    for feature, bit in enumerate(row["code 0"]):
        current = features[feature]
        value = mapping[current][bit]

        row_dict[current].append(value)

    row_dict["Code"].append(row["code 1"])
    row_dict["Explanation"].append(row["explanation B"])
        
    for feature, bit in enumerate(row["code 1"]):
        current = features[feature]
        value = mapping[current][bit]

        row_dict[current].append(value)

    return row_dict

for row in df.iterrows():
    output = code_to_list(row[1])
    for k, v in output.items():
        new_df[k].extend(v)

In [112]:
df_clean = pd.DataFrame.from_dict(new_df).drop_duplicates()

In [113]:
df_clean["Explanation"] = df_clean["Explanation"].apply(lambda x: BeautifulSoup(x, "lxml").text)
df_clean["Explanation"] = df_clean["Explanation"].str.replace("```", "").str.replace("html", "").str.replace("\n", " ").str.lstrip()

In [114]:
df_clean.head()

,Code,Explanation,Domain,Length,Structure,Formality,Detail,Persuasiveness
0,000000,Recommended Role: Cashier The Cashier position...,Low-stakes,Short,Running,Formal,Detailed,Persuasive
1,000001,Recommended Role: Cashier Your previous custom...,Low-stakes,Short,Running,Formal,Detailed,Decision-support
3,000010,Recommended Role: Cashier Your skills make you...,Low-stakes,Short,Running,Formal,Aggregated,Persuasive
5,000100,Recommended Role: Cashier Hey! The Cashier rol...,Low-stakes,Short,Running,Informal,Detailed,Persuasive
7,001000,Recommended Role: Cashier Your customer servi...,Low-stakes,Short,Bulleted,Formal,Detailed,Persuasive


In [115]:
def calcs(exps):
    print(f"   - Mean word count     : {exps.str.split().apply(len).mean()} ({exps.str.split().apply(len).std()})")
    print(f"   - Mean word length    : {exps.str.split().apply(lambda x : np.mean([len(i) for i in x])).mean()} ({exps.str.split().apply(lambda x : np.mean([len(i) for i in x])).std()})")
    print(f"   - Mean sentence length: {exps.str.split(".").apply(lambda x: np.mean([len(i.split()) for i in x])).mean()} ({exps.str.split(".").apply(lambda x: np.mean([len(i.split()) for i in x])).std()})")
    print()

def descriptive_checker(df, feature):
    options = df[feature].unique()

    for option in options:
        print(f"Descriptives for {option}:")
        calcs(df[df[feature] == option]["Explanation"])

    print()

for feature in ["Domain", "Length", "Structure", "Formality", "Detail", "Persuasiveness"]:
    descriptive_checker(df_clean, feature)

Descriptives for Low-stakes:
   - Mean word count     : 102.97297297297297 (42.375262822346)
   - Mean word length    : 5.52422004774462 (0.5676359496476335)
   - Mean sentence length: 15.55111075111075 (3.1795704886702594)

Descriptives for High-stakes:
   - Mean word count     : 105.34234234234235 (43.88093300895369)
   - Mean word length    : 5.622403562437031 (0.550442769224684)
   - Mean sentence length: 15.343408093408092 (3.45061768771275)


Descriptives for Short:
   - Mean word count     : 65.5625 (17.243648716342825)
   - Mean word length    : 5.586629317102307 (0.6029008398895579)
   - Mean sentence length: 13.368315972222225 (2.5946853379394117)

Descriptives for Long:
   - Mean word count     : 133.56349206349208 (32.17986849736861)
   - Mean word length    : 5.563165129272554 (0.5272212890042761)
   - Mean sentence length: 17.031216336573475 (2.9032230799883894)


Descriptives for Running:
   - Mean word count     : 118.76041666666667 (42.32126373390908)
   - Mean word le

In [116]:
import pandas as pd
import nltk
import spacy
from textblob import TextBlob
from textstat import flesch_reading_ease, flesch_kincaid_grade, gunning_fog
from nltk.corpus import wordnet as wn
from nltk import Tree
from collections import Counter
import numpy as np

# Load spaCy English model
nlp = spacy.load("en_core_web_sm")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\roans\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [117]:
def count_subordinate_clauses(doc):
    return sum(1 for token in doc if token.dep_ in ("advcl", "relcl", "csubj", "ccomp", "xcomp"))

def heylighen_formality(doc):
    tags = [token.tag_ for token in doc]
    pos_counts = Counter(tags)
    total = sum(pos_counts.values())
    formal = sum(pos_counts.get(tag, 0) for tag in ['NN', 'NNS', 'NNP', 'NNPS', 'JJ', 'IN', 'DT'])
    informal = sum(pos_counts.get(tag, 0) for tag in ['PRP', 'PRP$', 'RB', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ'])
    return 50 * ((formal - informal) / total + 1) if total > 0 else 0.0


# Apply feature extraction
def extract_features(row):
    text = row["Explanation"]
    doc = nlp(text)
    blob = TextBlob(text)
    
    sentences = nltk.sent_tokenize(text)
    words = nltk.word_tokenize(text)
    
    word_count = len(words)
    word_lengths = [len(w) for w in words if w.isalpha()]
    sentence_lengths = [len(nltk.word_tokenize(s)) for s in sentences]

    return pd.Series({
        "word_count": word_count,
        "avg_word_length": np.mean(word_lengths) if word_lengths else 0,
        "avg_sentence_length": np.mean(sentence_lengths) if sentence_lengths else 0,
        "gunning_fog": gunning_fog(text),
        "sentiment_polarity": blob.sentiment.polarity,
        "sentiment_subjectivity": blob.sentiment.subjectivity,
        "subordinate_clauses": count_subordinate_clauses(doc),
        "formality_score": heylighen_formality(doc),
    })

# Apply to the DataFrame
df_features = df_clean.apply(extract_features, axis=1)

# Combine with original
df_enriched = pd.concat([df_clean, df_features], axis=1)

In [118]:
 import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import statsmodels.api as sm

data = [
    ("Need_for_Cognition:Detail", 0.841, "Detail"),
    ("Need_for_Cognition:Formality", 1.348, "Formality"),
    ("Need_for_Cognition:Length", 0.535, "Length"),
    ("Need_for_Cognition:Persuasiveness", 0.014, "Persuasiveness"),
    ("Need_for_Cognition:Structure", -2.737, "Structure"),
    
    ("Need_for_closure:Detail", -2.751, "Detail"),
    ("Need_for_closure:Formality", -0.738, "Formality"),
    ("Need_for_closure:Length", -2.401, "Length"),
    ("Need_for_closure:Persuasiveness", -0.662, "Persuasiveness"),
    ("Need_for_closure:Structure", 6.553, "Structure"),
    
    ("Susceptibility_to_persuasion:Detail", -0.992, "Detail"),
    ("Susceptibility_to_persuasion:Formality", -1.884, "Formality"),
    ("Susceptibility_to_persuasion:Length", -0.185, "Length"),
    ("Susceptibility_to_persuasion:Persuasiveness", 1.677, "Persuasiveness"),
    ("Susceptibility_to_persuasion:Structure", 1.383, "Structure"),
    
    ("Skepticism:Detail", 3.332, "Detail"),
    ("Skepticism:Formality", 1.882, "Formality"),
    ("Skepticism:Length", 2.358, "Length"),
    ("Skepticism:Persuasiveness", 0.606, "Persuasiveness"),
    ("Skepticism:Structure", -8.178, "Structure"),
    
    ("AI_Expertise:Detail", -1.569, "Detail"),
    ("AI_Expertise:Formality", -0.992, "Formality"),
    ("AI_Expertise:Length", -0.749, "Length"),
    ("AI_Expertise:Persuasiveness", -0.476, "Persuasiveness"),
    ("AI_Expertise:Structure", 3.856, "Structure"),
]


# Convert to DataFrame
interactions_df = pd.DataFrame(data, columns=["interaction", "coefficient", "dimension"])

In [119]:
# List of features to average
features = ["word_count", "avg_word_length", "avg_sentence_length", 
            "gunning_fog", "sentiment_polarity", "sentiment_subjectivity",
            "subordinate_clauses", "formality_score"
]

# Example: grouping by Detail level (Detailed vs Aggregated)
grouped_detail = df_enriched.groupby("Detail")[features].mean().reset_index()
print(grouped_detail)

       Detail  word_count  avg_word_length  avg_sentence_length  gunning_fog  \
0  Aggregated  111.087302         5.382220            19.297831    14.986934   
1    Detailed  133.791667         5.438871            20.543579    15.522532   

   sentiment_polarity  sentiment_subjectivity  subordinate_clauses  \
0            0.239371                0.502699             6.666667   
1            0.185037                0.480172             6.906250   

   formality_score  
0        64.323932  
1        67.867191  


In [120]:
# List of dimensions you're analyzing
dimensions = ["Detail", "Formality", "Length", "Structure", "Persuasiveness"]

# Collect difference vectors per design dimension
dimension_diffs = {}

for dim in dimensions:
    grouped = df_enriched.groupby(dim)[features].mean()
    
    if grouped.shape[0] != 2:
        print(f"Skipping {dim}: not binary")
        continue

    # Compute difference between high and low (order will depend on groupby key sort)
    diff = grouped.diff().iloc[-1]  # assumes row order: [Low, High] → diff = High - Low
    dimension_diffs[dim] = diff

df_diffs = pd.DataFrame(dimension_diffs).T  # shape: (5 dimensions) × (n features)
df_diffs.index.name = "dimension"
df_diffs.reset_index(inplace=True)


In [121]:
df_interactions = pd.DataFrame(data).rename(columns = {0: "interaction", 1: "coefficient", 2: "dimension"})
df_interactions.head()

,interaction,coefficient,dimension
0,Need_for_Cognition:Detail,0.841,Detail
1,Need_for_Cognition:Formality,1.348,Formality
2,Need_for_Cognition:Length,0.535,Length
3,Need_for_Cognition:Persuasiveness,0.014,Persuasiveness
4,Need_for_Cognition:Structure,-2.737,Structure


In [122]:
# Assuming you have this already:
# df_interactions = DataFrame with ['interaction', 'coefficient', 'dimension']

# Merge in design feature diffs
df_full = df_interactions.merge(df_diffs, on="dimension", how="left")

from scipy.stats import pearsonr

for feature in features:
    corr, p = pearsonr(df_full["coefficient"], df_full[feature])
    print(f"Correlation between interaction coefficient and Δ{feature}: r = {corr:.3f}, p = {p:.3f}")


Correlation between interaction coefficient and Δword_count: r = 0.004, p = 0.984
Correlation between interaction coefficient and Δavg_word_length: r = 0.013, p = 0.953
Correlation between interaction coefficient and Δavg_sentence_length: r = 0.020, p = 0.923
Correlation between interaction coefficient and Δgunning_fog: r = 0.022, p = 0.915
Correlation between interaction coefficient and Δsentiment_polarity: r = 0.031, p = 0.882
Correlation between interaction coefficient and Δsentiment_subjectivity: r = 0.038, p = 0.857
Correlation between interaction coefficient and Δsubordinate_clauses: r = 0.008, p = 0.971
Correlation between interaction coefficient and Δformality_score: r = -0.027, p = 0.900


In [123]:
df_full["trait"] = df_full["interaction"].apply(lambda x: x.split(":")[0])
traits = df_full["trait"].unique()

for trait in traits:
    print(f"\nTrait: {trait}")
    subset = df_full[df_full["trait"] == trait]
    for feature in features:
        corr, p = pearsonr(subset["coefficient"], subset[feature])
        if abs(corr) > 0.4:
            print(f"  Correlation with Δ{feature}: r = {corr:.3f}, p = {p:.3f}")



Trait: Need_for_Cognition
  Correlation with Δavg_sentence_length: r = -0.669, p = 0.217
  Correlation with Δgunning_fog: r = -0.574, p = 0.312

Trait: Need_for_closure
  Correlation with Δword_count: r = 0.490, p = 0.402
  Correlation with Δavg_sentence_length: r = 0.618, p = 0.267
  Correlation with Δsubordinate_clauses: r = 0.546, p = 0.341

Trait: Susceptibility_to_persuasion
  Correlation with Δavg_word_length: r = 0.658, p = 0.227
  Correlation with Δavg_sentence_length: r = 0.450, p = 0.448
  Correlation with Δgunning_fog: r = 0.754, p = 0.141
  Correlation with Δsentiment_subjectivity: r = 0.495, p = 0.396

Trait: Skepticism
  Correlation with Δword_count: r = -0.422, p = 0.479
  Correlation with Δavg_sentence_length: r = -0.622, p = 0.262
  Correlation with Δsubordinate_clauses: r = -0.453, p = 0.444

Trait: AI_Expertise
  Correlation with Δavg_sentence_length: r = 0.581, p = 0.304
